# run the notebook in google colab

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [2]:
# upload the train.csv file to the required folder and give the folder id in the below line
folder_id = '********************'
list2 = drive.ListFile({'q': "'%s' in parents" % folder_id}).GetList()
for file in list2:
  print('title {}, id {}'.format(file['title'], file['id']))

title train.zip, id 1AtKA7KlxxtYIjcO2QX68i9FhVJeazaKK


In [0]:
# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '******************************'  # give the file id obtined from the code in above cell
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('train.zip')

In [4]:
!unzip train.zip

Archive:  train.zip
  inflating: train.csv               


In [0]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [6]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2019-10-07 04:13:37--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2019-10-07 04:13:38--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2019-10-07 04:13:38--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [7]:
!unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [0]:
import os
import sys
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import Constant

In [9]:
import numpy as np
import pandas as pd
print('Indexing word vectors.')

embeddings_index = {}
with open('glove.6B.100d.txt') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, 'f', sep=' ')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

Indexing word vectors.
Found 400000 word vectors.


In [0]:
full_data = pd.read_csv('train.csv')

In [11]:
full_data.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [12]:
full_data.shape

(1306122, 3)

In [0]:
#dropping the column qid
full_data.drop(['qid'],inplace=True,axis=1)

In [14]:
full_data.head()

,question_text,target
0,How did Quebec nationalists see their province...,0
1,"Do you have an adopted dog, how would you enco...",0
2,Why does velocity affect time? Does velocity a...,0
3,How did Otto von Guericke used the Magdeburg h...,0
4,Can I convert montra helicon D to a mountain b...,0


In [0]:
x=full_data['question_text']
y=full_data['target']

In [16]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(x)
sequences = tokenizer.texts_to_sequences(x)

In [18]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

Found 222161 unique tokens.


In [0]:
y= np.array(y).reshape(-1,1)

In [0]:
labels = to_categorical(np.asarray(y))

In [21]:
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', y.shape)

Shape of data tensor: (1306122, 1000)
Shape of label tensor: (1306122, 1)


In [0]:
# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

In [23]:
print('Preparing embedding matrix.')

# prepare embedding matrix
num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

Preparing embedding matrix.


In [0]:
# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [26]:
print('Training model.')

# train a 1D convnet with global maxpooling
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(64, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(64, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(64, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(64, activation='relu')(x)
# preds = Dense(len(labels_index), activation='softmax')(x)
preds = Dense(2, activation='softmax')(x)

Training model.


In [0]:
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [28]:
model.fit(x_train, y_train,
          batch_size=128,
          epochs=5,
          validation_data=(x_val, y_val))

Train on 1044898 samples, validate on 261224 samples
Epoch 1/5
1044898/1044898 [==============================] - 282s 270us/sample - loss: 0.1397 - acc: 0.9489 - val_loss: 0.1312 - val_acc: 0.9512
Epoch 2/5
1044898/1044898 [==============================] - 247s 237us/sample - loss: 0.1324 - acc: 0.9511 - val_loss: 0.1287 - val_acc: 0.9517
Epoch 3/5
1044898/1044898 [==============================] - 247s 237us/sample - loss: 0.1313 - acc: 0.9520 - val_loss: 0.1401 - val_acc: 0.9481
Epoch 4/5
1044898/1044898 [==============================] - 248s 237us/sample - loss: 0.1298 - acc: 0.9525 - val_loss: 0.1399 - val_acc: 0.9502
Epoch 5/5
1044898/1044898 [==============================] - 247s 237us/sample - loss: 0.1298 - acc: 0.9527 - val_loss: 0.1582 - val_acc: 0.9418


In [0]:
y_pred = model.predict(x_val)

In [0]:
from sklearn.metrics import roc_auc_score

In [31]:
roc_auc_score(y_val,y_pred)

0.9312906562942552

In [0]:
cutoffs=np.linspace(0.01,0.99,99)
real  = y_val[:,1]
y_predicted = y_pred[:,1]

In [0]:
KS_all=[]

for cutoff in cutoffs:
    
    predicted=(y_predicted>cutoff).astype(int)

    TP=((predicted==1) & (real==1)).sum()
    TN=((predicted==0) & (real==0)).sum()
    FP=((predicted==1) & (real==0)).sum()
    FN=((predicted==0) & (real==1)).sum()
    
    P=TP+FN
    N=TN+FP
      
    KS=(TP/P)-(FP/N)
    
    
    KS_all.append(KS)

In [41]:
mycutoff=cutoffs[KS_all==max(KS_all)][0]
mycutoff

0.17

In [0]:
hard_classes = (y_predicted>mycutoff).astype(int)

In [47]:
hard_classes  ## 1- spam, 0- not spam

array([0, 1, 0, ..., 0, 0, 1])